<h2>Residual Network using Keras</h2>

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc

import h5py

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform,  glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

%matplotlib inline

2024-03-04 20:15:00.795765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 20:15:00.795892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 20:15:00.876847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 20:15:01.042268: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 20:15:02.380604: W tensorflow/compiler/tf2

<h3>Identity Block</h3>

In [2]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    F1, F2, F3 = filters

    X_shortcut = X
    cache = []

    X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=f, strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    # add X_shortcut to the result of last layer
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

<h5>Testing Identity Block</h5>

In [3]:
np.random.seed(1)

# making input data for testing
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis=0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3], initializer=lambda seed=0:constant(value=1), training=True)
A3np = A3.numpy()
print(np.around(A3np[:,(0,-1),:,:].mean(axis = 3), 5))

2024-03-04 20:15:04.731712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-04 20:15:05.014467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-04 20:15:05.014525: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-04 20:15:05.016980: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-04 20:15:05.017039: I external/local_xla/xla/stream_executor

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40736 0.40736 0.40736 0.40736]
  [0.40736 0.40736 0.40736 0.40736]]

 [[5.      5.      5.      3.25949]
  [3.25949 3.25949 3.25949 2.40736]]]


<h3>Convolution Block</h3>

In [4]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=f, strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    X_shortcut = Conv2D(filters=F3, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    X = Add()([X_shortcut, X])

    X = Activation('relu')(X)

    return X


<h4>Testing convolutional block</h4>

In [5]:
np.random.seed(1)

X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6])
print(A[0])

tf.Tensor(
[[[0.        2.7823162 0.        0.        1.6960442 2.8218517]
  [0.        1.5445004 0.        0.        2.170656  1.3908148]]

 [[0.        1.9399529 0.        0.        1.4798119 1.9157798]
  [0.        0.        0.        0.9879823 1.123416  0.       ]]], shape=(2, 2, 6), dtype=float32)


<h3>RESNET50 Implementation</h3>

In [6]:
def ResNet50(input_shape = (64, 64, 3), classes=6):
    X_input = Input(input_shape)

    X = ZeroPadding2D(padding=(3, 3))(X_input)

    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, s=1, filters=[64, 64, 256])
    X = identity_block(X, f=3, filters=[64, 64, 256])
    X = identity_block(X, f=3, filters=[64, 64, 256])

    X = convolutional_block(X, f=3, s=2, filters=[128, 128, 512])
    X = identity_block(X, f=3, filters=[128, 128, 512])
    X = identity_block(X, f=3, filters=[128, 128, 512])
    X = identity_block(X, f=3, filters=[128, 128, 512])

    X = convolutional_block(X, f=3, s=2, filters=[256, 256, 1024])    
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])

    X = convolutional_block(X, f=3, s=2, filters=[512, 512, 2048])
    X = identity_block(X, f=3, filters=[512, 512, 2048])
    X = identity_block(X, f=3, filters=[512, 512, 2048])

    X = AveragePooling2D((2, 2))(X)

    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=glorot_uniform(seed=0))(X)

    model = Model(inputs=X_input, outputs=X)
    return model

In [7]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
## print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

<h4>Load data</h4>

In [8]:
def load_dataset():
    train_dataset = h5py.File('../datasets/train_signs.h5', 'r')
    train_X_OG = np.array(train_dataset['train_set_x'][:])
    train_Y_OG = np.array(train_dataset['train_set_y'][:])

    test_dataset = h5py.File('../datasets/test_signs.h5', 'r')
    test_X_OG = np.array(test_dataset['test_set_x'][:])
    test_Y_OG = np.array(test_dataset['test_set_y'][:])

    classes = np.array(test_dataset['list_classes'][:])

    train_Y_OG = train_Y_OG.reshape((1, train_Y_OG.shape[0]))
    test_Y_OG = test_Y_OG.reshape((1, test_Y_OG.shape[0]))

    return train_X_OG, train_Y_OG, test_X_OG, test_Y_OG, classes

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

<h3>Training</h3>

In [10]:
train_X_OG, train_Y_OG, test_X_OG, test_Y_OG, classes = load_dataset()

X_train = train_X_OG/255.
X_test=  test_X_OG/255.

Y_train = convert_to_one_hot(train_Y_OG, 6).T
Y_test = convert_to_one_hot(test_Y_OG, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

model.fit(X_train, Y_train, epochs = 15, batch_size = 16)

preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)
Epoch 1/15
68/68 [==============================] - 4s 54ms/step - loss: 0.0456 - accuracy: 0.9870
Epoch 2/15
68/68 [==============================] - 3s 50ms/step - loss: 0.0251 - accuracy: 0.9926
Epoch 3/15
68/68 [==============================] - 3s 50ms/step - loss: 0.2215 - accuracy: 0.9537
Epoch 4/15
68/68 [==============================] - 3s 48ms/step - loss: 0.9976 - accuracy: 0.7722
Epoch 5/15
68/68 [==============================] - 3s 48ms/step - loss: 0.2321 - accuracy: 0.9259
Epoch 6/15
68/68 [==============================] - 3s 50ms/step - loss: 0.0796 - accuracy: 0.9750
Epoch 7/15
68/68 [==============================] - 3s 48ms/step - loss: 0.0372 - accuracy: 0.9889
Epoch 8/15
68/68 [==============================] - 3s 50ms/step - loss: 0.0410 - accuracy: 0.9870
Epoch 9/15
68/68 [=============